# 1.0 Recommender system - warm up

In [35]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process                      # matches strings even though not completly same

In [3]:
df_movies = pd.read_csv('../lab/data/movies.csv', usecols=['movieId', 'title'], dtype={'movieID':'int32', 'title': 'str'})
df_ratings = pd.read_csv('../lab/data/ratings.csv', usecols=['userId', 'movieId', 'rating'], dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [49]:
df_movies.head(3)

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)


In [50]:
df_ratings.head(3)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0


In [51]:
# creating a pivot table with NaNs and replcing NaN with 0 by fillna(0)
movies_users = df_ratings.pivot(index='movieId', columns='userId', values='rating').fillna(0)
movies_users.head(3)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0


In [12]:
movies_users.values

# each row is a vector

array([[4. , 0. , 0. , ..., 2.5, 3. , 5. ],
       [0. , 0. , 0. , ..., 2. , 0. , 0. ],
       [4. , 0. , 0. , ..., 2. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]], dtype=float32)

In [ ]:
# crating a sparse matrix
# indexes position in table (row, col) and gives value for all non-zero entries
matrix_movies_users = csr_matrix(movies_users.values) # same result with just (movies_users)
print(type(matrix_movies_users))
print(matrix_movies_users.shape)
print(matrix_movies_users[0]) 

In [32]:
# create model:
# metrics: distance (types: Euclidean, Manhattan, Minkowski) or cosine similarity between 2 vectors (rows) -> 0 - 1 where 1 is totally equal
# algoritm: brute compares one vector with all other in dataset
# guessing 20 neighbours

model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20)

In [34]:
# train model

print(model_knn.fit(matrix_movies_users))

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)


In [53]:
# create function recommender(movie_name) -> list of recommended movies
# data is matrix_movies_users
# idx is index: fuzzywuzzy process used

def recommender(movie_name):
    idx = process.extractOne(movie_name, df_movies['title'])
    print(idx)

recommender('grumpier old men')

('Grumpier Old Men (1995)', 95, 2)


In [54]:
df_movies['title'][2]

'Grumpier Old Men (1995)'

In [69]:
def recommender(movie_name, data, model, n_recommendations):
    model.fit(data)
    idx = process.extractOne(movie_name, df_movies['title'])[2] # to get 2 in example for 'Ghost, index in df_movies
    print(f'Movie selected:  {df_movies["title"][idx]};  Index:  {idx}')
    print('Searching for recommendations....')
    distances, indices = model.kneighbors(data[idx], n_neighbors = n_recommendations) # distances is cos similarity
    print(distances, indices)
    print([df_movies["title"][i].where(i!=idx) for i in indices]) # list of movie recommendations based on indices; != skip incex for movie_name

recommender('grumpier old men', matrix_movies_users, model_knn, 5)


Movie selected:  Grumpier Old Men (1995);  Index:  2
Searching for recommendations....
[[4.1723251e-07 5.5201328e-01 5.5321550e-01 5.5491757e-01 5.6375337e-01]] [[   2 2578  607  622  594]]
[2                               NaN
2578         Good Earth, The (1937)
607               Striptease (1996)
622     Nutty Professor, The (1996)
594                  Twister (1996)
Name: title, dtype: object]


In [70]:
print(f' {df_movies["title"][2]}')

 Grumpier Old Men (1995)
